In [27]:
import os
print(os.getcwd())

/home/jupyter/work/resources


## Загрузка данных в ClickHouse

## Подключение и настройка

https://cloud.yandex.ru/docs/managed-clickhouse/ (см. слайды)

### Данные для доступа
* Из интерфейса облака в разделе хосты копируем имя хост в переменную CH_HOST_NAME
* Используем заведенного юзера в переменной CH_USER
* Используем заведенного юзера в переменной CH_USER
* Сохраним пароль заведенного пользователя в текстовый файл .chpass.txt
* В переменную CH_PASS считаем содержимое файла .chpass.txt
* В переменную cacert поместим путь к сертификату для подключения к серверу. Файл YandexInternalRootCA.crt должен лежать в репозитории. Скачать можно, выполнив команды:

`#!:bash
mkdir -p <some-dir>
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" -O <some-dir>/YandexInternalRootCA.crt
chmod 655 <some-dir>/YandexInternalRootCA.crt`

In [122]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1c-dp4f585c5tk96k9m.mdb.yandexcloud.net'
CH_USER      = 'firstsvet02'
CH_DB_NAME   = 'AppMetrica_workshop'

#-------------------------------------------
CH_PASS      = open('chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'certificate/YandexInternalRootCA.crt'

## Функции для интеграции с ClickHouse
В файле `utils` есть класс `simple_ch_client` для работы с ClickHouse
Сначала надо создать экземпляр класса, инициализировав его начальными параметрами - хост, пользователь, пароль и путь к сертификату `simple_ch_client(CH_HOST, CH_USER, CH_PASS, cacert)`
В классе есть 4 метода:
* .get_version() - получает текущую версию ClickHouse. Хороший способ проверить, что указанные при инициализации параметры работают
* .get_clickhouse_data(query) - выполняет запрос query и возвращает результат в текстовом формате
* .get_clickhouse_df(query) - выполняет запрос query и возвращает результат в виде DataFrame
* .upload(table, content) - загружает таблицу content, которая подается в текстовом формате в таблицу ClickHouse'а с именем table


## Проверяем ClickHouse
Используя заговленные выше переменные проверим доступ до сервера (как в документации https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#connection-string) Этот метод реализован в методе `.get_version()` класса для работы с ClickHouse При успешном подключении не произойдет никакой ошибки при выполнении этого метода, и он сам вернет версию сервера ClickHouse (например 21.3.2.5)

In [118]:
import utils
from utils import simple_ch_client

In [119]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

In [120]:
my_client.get_version()

21.8.7.22



## Загружаем данные

In [128]:
import pandas as pd
import requests
import json

In [129]:
raw_df = pd.read_csv('raw_auto_data.csv', sep = ',')

In [130]:
print (raw_df.shape)
raw_df.head(2)

(7812901, 5)


,ym:pv:date,session_id,appmetrica_device_id,event_name,event_json
0,2020-10-21,13009834383587784298,9179943992052001801,SNIPPET_SHOW,"{'body_type': 'ALLROAD_3_DOORS', 'mark': 'VAZ'..."
1,2020-10-17,3706642344325836439,16017347114857937401,SNIPPET_SHOW,"{'body_type': 'ALLROAD_3_DOORS', 'mark': 'VAZ'..."


### Преобразовываем

In [60]:
for key in ['body_type', 'mark', 'section']:
    raw_df[key] = list(map(lambda x: json.loads(str(x).replace("'",'"'))[key], raw_df['event_json']))

In [65]:
raw_df.rename(columns={'ym:pv:date':'date'}, inplace = True)

In [126]:
q = f'drop table if exists {CH_DB_NAME}.raw_auto_data '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.raw_auto_data (
    date Date,
    appmetrica_device_id UInt64,
    session_id UInt64,
    event_name String,
    mark String,
    body_type String,
    section String 
) ENGINE = MergeTree(date, intHash64(appmetrica_device_id), (date, intHash64(appmetrica_device_id)), 8192)
'''

my_client.get_clickhouse_data(q)

''

In [127]:
my_client.upload(
    f'{CH_DB_NAME}.raw_auto_data',
    raw_df[['date', 'appmetrica_device_id', 'session_id', 'event_name',
        'mark', 'body_type', 'section']].to_csv(index = False, sep = '\t'))

''